In [1]:
NUMPY_EXPERIMENTAL_ARRAY_FUNCTION = 1  # FIXME is this needed?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import cupy
import cupy as cp
import matplotlib.pyplot as plt
from string import Template
from math import sin, cos, atan2, sqrt

In [4]:
import numba
from numba import jit, njit, prange

In [5]:
print(cupy.cuda.Device())

<CUDA Device 0>


In [6]:
# "vectors" below are 2d numpy arrays of shape (3, number_of_vectors)


def mag(v):
    return np.linalg.norm(v, axis=0)


def unit(v):
    return v / np.linalg.norm(v, axis=0)


# # the following code seems 10% faster for cpu, but similar for gpu
# def pwr(v):
#     return v[0, :]**2 + v[1, :]**2 + v[2, :]**2


# def mag(v):
#     return np.sqrt(pwr(v))


# def unit(v):
#     return v / mag(v)


def inner(v1, v2):
    #     return v1[0, :]*v2[0, :] + v1[1, :]*v2[1, :] + v1[2, :]*v2[2, :]
    return np.sum(v1 * v2, axis=0)


def cross(v1, v2):
    return np.cross(v1, v2, axisa=0, axisb=0, axisc=0)


# the "kernel"
def kernel(x1, x2, v1, v2):
    dv = v1 - v2
    v_avg = 0.5 * (v1 + v2)
    dv_perp = cross(v_avg, cross(dv, v_avg))

    l_unit = unit(v_avg)
    m_unit = unit(dv_perp)
    n_unit = cross(l_unit, m_unit)

    dx = x1 - x2
    l = inner(l_unit, dx)
    m = inner(m_unit, dx)
    n = inner(n_unit, dx)

    result = (mag(dv))**2
    
    return l, m, n, result


In [7]:
Nsamples = int(1e6)
xyz1 = np.random.random((3, Nsamples))
xyz2 = np.random.random((3, Nsamples))
vxyz1 = np.random.random((3, Nsamples))
vxyz2 = np.random.random((3, Nsamples))

In [8]:
%timeit l, m, n, z = kernel(xyz1, xyz2, vxyz1, vxyz2)

243 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
xyz1_gpu = cupy.asarray(xyz1)
xyz2_gpu = cupy.asarray(xyz2)
vxyz1_gpu = cupy.asarray(vxyz1)
vxyz2_gpu = cupy.asarray(vxyz2)

In [10]:
%timeit l, m, n, z = kernel(xyz1_gpu, xyz2_gpu, vxyz1_gpu, vxyz2_gpu)

3.39 ms ± 525 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [65]:
# an application using cross-product between two vectors, 
# some vector norm and arithmetic calculations, and a summation
def biot_savart_dst(x2,j2,vol2,r_mag=2.5):
# calculate magnetic perturbation at coordinate system origin (0,0,0)
# x2,j2 = 3,N array of vectors
# vol2 (N) array of scalar volumes at position x2 with currents j2
    n_x2=len(x2[0,:])
    x2_mag=mag(x2)
    dst_factor=1.e-7
    x2_mag3=x2_mag*x2_mag*x2_mag
                           
    vol_x2mag3=(np.reshape(np.repeat(vol2/x2_mag,3),(n_x2,3))).T

    dst=cross(j2,x2)*vol_x2mag3  # (3,N) contributions to dst, a (3) vector
    dst_sum=dst_factor*np.sum(dst,axis=1)   # (3,N) -> (3)

    return dst_sum  # single vector value

def biot_savart_sum(x1,x2,j2,vol2):
# calculate magnetic perturbation at a position (x1) away from the origin 
# x1,x2,j2 = 3,N array of vectors
# vol2 (M) array of scalar volumes at position x2 with currents j2
# x1 (3,M) array of positions of observer locations
    n_x1=len(x1[0,:])
    n_x2=len(x2[0,:])
    b1_sum=np.zeros((3,n_x1))  # might optimize of this array were in GPU
    for i in range(n_x1): # M iterations
        dx=x2-np.reshape(np.repeat(x1[:,i],n_x2),(3,n_x2))
        b_=biot_savart_dst(dx,j2,vol2)   # this is the heavy computation step
        if hasattr(b_,'get'):     # get() needed to get data from GPU to CPU
            b1_sum[:,i]=b_.get()
        else:
            b1_sum[:,i]=b_
        
    return b1_sum  # (3,M) values


In [12]:
Nsamples = int(1e3)
Msamples = int(100)
x1 = np.random.random((3, Msamples))+1
x2 = np.random.random((3, Nsamples))+3
j2 = np.random.random((3, Nsamples))
vol2 = np.random.random((Nsamples))+0.5
x1_gpu = cupy.asarray(x1)
x2_gpu = cupy.asarray(x2)
j2_gpu = cupy.asarray(j2)
vol2_gpu = cupy.asarray(vol2)

In [13]:
%timeit dst = biot_savart_dst(x2, j2, vol2)

138 µs ± 485 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%timeit dst2 = biot_savart_dst(x2_gpu, j2_gpu, vol2_gpu)

381 µs ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [116]:
Nsamples = int(1e5)
Msamples = int(100)
x1 = np.random.random((3, Msamples))+1
x2 = np.random.random((3, Nsamples))+3
j2 = np.random.random((3, Nsamples))
vol2 = np.random.random((Nsamples))+0.5
x1_gpu = cupy.asarray(x1)
x2_gpu = cupy.asarray(x2)
j2_gpu = cupy.asarray(j2)
vol2_gpu = cupy.asarray(vol2)

In [117]:
%timeit dst = biot_savart_dst(x2, j2, vol2)

4.52 ms ± 7.13 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [135]:
%timeit b1 = biot_savart_sum(x1,x2, j2, vol2)

564 ms ± 725 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:
%timeit b1 = biot_savart_sum(x1_gpu,x2_gpu, j2_gpu, vol2_gpu)

57.9 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
Nsamples = int(1e6)
#Msamples = int(100)
#x1 = np.random.random((3, Msamples))+1
x2 = np.random.random((3, Nsamples))+3
j2 = np.random.random((3, Nsamples))
vol2 = np.random.random((Nsamples))+0.5
x1_gpu = cupy.asarray(x1)
x2_gpu = cupy.asarray(x2)
j2_gpu = cupy.asarray(j2)
vol2_gpu = cupy.asarray(vol2)

In [69]:
%timeit dst = biot_savart_dst(x2, j2, vol2)

75.8 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [70]:
%timeit dst2 = biot_savart_dst(x2_gpu, j2_gpu, vol2_gpu)

2.07 ms ± 101 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [66]:
Nsamples = int(1e8)
Msamples = int(100)
x1 = np.random.random((3, Msamples))+1
x2 = np.random.random((3, Nsamples))+3
j2 = np.random.random((3, Nsamples))
vol2 = np.random.random((Nsamples))+0.5
x1_gpu = cupy.asarray(x1)
x2_gpu = cupy.asarray(x2)
j2_gpu = cupy.asarray(j2)
vol2_gpu = cupy.asarray(vol2)

In [67]:
%timeit dst = biot_savart_dst(x2, j2, vol2)

10.1 s ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%timeit dst2 = biot_savart_dst(x2_gpu, j2_gpu, vol2_gpu)

202 ms ± 84.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
# total memory used
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()
print(mempool.used_bytes())              # 5600010752
print(mempool.total_bytes())             # 16000010752
print(pinned_mempool.n_free_blocks())    # 4

5600010752
16000010752
2


In [44]:
# free memory used by calculations (the gpu input arrays are still there)
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 5600010752 after only running 1e8
print(mempool.total_bytes())             # 5600010752
print(pinned_mempool.n_free_blocks())    # 0

5600010752
5600010752
0


In [45]:
x1_gpu = None
x2_gpu = None
j2_gpu = None
vol2_gpu = None
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 8192 - 8 kBytes?
print(mempool.total_bytes())             # 8192
print(pinned_mempool.n_free_blocks())    # 0

8192
8192
0


In [17]:
import os
HOME=os.environ['HOME']
filename=HOME+'/SWMF_MAG_DATA/Michael_Madelaire_082120_1/3d__var_1_e19980423-180000-000.out'
print(filename)

/home/u00u55abwyjsQnkErP357/SWMF_MAG_DATA/Michael_Madelaire_082120_1/3d__var_1_e19980423-180000-000.out


In [18]:
import numpy as np
import pandas as pd
import os
import scipy
import scipy.constants as constants
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
import math
import time
import kamodo

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot, plot
from plotly.subplots import make_subplots

/home/u00u55abwyjsQnkErP357/.local/lib/python3.6/site-packages/sympy/physics/units/quantities.py:81: SymPyDeprecationWarning: 

Moving method to UnitSystem class has been deprecated since SymPy 1.5.
Use unit_system.set_quantity_dimension or
nanotesla.set_global_relative_scale_factor instead. See
https://github.com/sympy/sympy/issues/17765 for more info.

  useinstead="unit_system.set_quantity_dimension or {}.set_global_relative_scale_factor".format(self),
/home/u00u55abwyjsQnkErP357/.local/lib/python3.6/site-packages/sympy/physics/units/quantities.py:92: SymPyDeprecationWarning: 

Moving method to UnitSystem class has been deprecated since SymPy 1.5.
Use unit_system.set_quantity_scale_factor or
nanotesla.set_global_relative_scale_factor instead. See
https://github.com/sympy/sympy/issues/17765 for more info.

  useinstead="unit_system.set_quantity_scale_factor or {}.set_global_relative_scale_factor".format(self),
/home/u00u55abwyjsQnkErP357/.local/lib/python3.6/site-packages/sympy/physi

In [28]:
from swmf_gm import SWMF_GM

In [29]:
HOME=os.environ['HOME']
runpath=HOME+'/SWMF_MAG_DATA/'
runname='Michael_Madelaire_082120_1'
file=runpath+runname+'/3d__var_1_e19980423-180000-000.out'

swmf = SWMF_GM(file, runpath=runpath, runname=runname)

opening /home/u00u55abwyjsQnkErP357/SWMF_MAG_DATA/Michael_Madelaire_082120_1/3d__var_1_e19980423-180000-000.out
... found 19 variables
... setting filetime =  1000/01/01 00:00:00 UT
... file grid has 1007616 cells with minimum dx = 0.125
Time loading file and precomputing default interpolations: 15.4038 seconds


In [30]:
swmf.variables

{'x': {'units': 'R',
  'data': array([-220., -212., -220., ...,   30.,   26.,   30.], dtype=float32),
  'interpolator': array([-50. , -49.5, -49. , ...,  19. ,  19.5,  20. ])},
 'y': {'units': 'R',
  'data': array([-124., -124., -116., ...,  122.,  126.,  126.], dtype=float32),
  'interpolator': array([-20., -20., -20., ...,  20.,  20.,  20.])},
 'z': {'units': 'R',
  'data': array([-124., -124., -124., ...,  126.,  126.,  126.], dtype=float32),
  'interpolator': array([0., 0., 0., ..., 0., 0., 0.])},
 'rho': {'units': 'Mp/cc',
  'data': array([11.199, 11.199, 11.199, ..., 11.199, 11.199, 11.199], dtype=float32),
  'interpolator': array([ 2.18929994,  2.20574248,  2.22218502, ..., 11.19900036,
         11.19900036, 11.19900036])},
 'ux': {'units': 'km/s',
  'data': array([-322.443, -322.443, -322.443, ..., -322.443, -322.443, -322.443],
        dtype=float32),
  'interpolator': array([-160.99200439, -160.083004  , -159.1740036 , ..., -322.44299316,
         -322.44299316, -322.44299316

In [31]:
swmf

/home/u00u55abwyjsQnkErP357/.local/lib/python3.6/site-packages/sympy/core/relational.py:490: SymPyDeprecationWarning:



Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.




SWMF_GM([(x(xvec),
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (x,
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (y(xvec),
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (y,
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (z(xvec),
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (z,
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (rho(xvec),
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (rho,
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (ux(xvec),
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xvec)>),
         (ux,
          <function swmf_gm.SWMF_GM.register_variable.<locals>.interpolate(xve

In [36]:
jvec=np.vstack([swmf.variables['jx']['data'],swmf.variables['jy']['data'],swmf.variables['jz']['data']])

In [37]:
jvec.shape

(3, 1007616)

In [38]:
xvec=np.vstack([swmf.variables['x']['data'],swmf.variables['y']['data'],swmf.variables['z']['data']])


In [39]:
xvec.shape

(3, 1007616)

In [40]:
nx1=101
lats=np.array(np.arange(nx1)*math.pi/(nx1-1.),dtype='float32')
x1vec=np.vstack([np.cos(lats),np.zeros(nx1),np.sin(lats)])

In [41]:
x1vec.shape

(3, 101)

In [42]:
timeit dst=biot_savart_dst(xvec,jvec,vol)

41.4 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
xvec_gpu=cp.array(xvec)
jvec_gpu=cp.array(jvec)
x1vec_gpu=cp.array(x1vec)
vol_gpu=cp.array(vol)

In [47]:
timeit dst=biot_savart_dst(xvec_gpu,jvec_gpu,vol_gpu)

397 µs ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
dst_gpu=biot_savart_dst(xvec_gpu,jvec_gpu,vol_gpu)

In [64]:
dst_gpu.get()

array([-0.00765572,  0.00644158, -0.0015901 ], dtype=float32)

In [55]:
timeit deltab=biot_savart_sum(x1vec,xvec,jvec,vol)

8.78 s ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
timeit deltab=biot_savart_sum(x1vec_gpu,xvec_gpu,jvec_gpu,vol_gpu)

226 ms ± 872 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
deltab_gpu=biot_savart_sum(x1vec_gpu,xvec_gpu,jvec_gpu,vol_gpu)

In [62]:
deltab_gpu.T

array([[-0.00796291,  0.00642199, -0.00138094],
       [-0.00798377,  0.0064346 , -0.00136993],
       [-0.00800419,  0.00644709, -0.00135905],
       [-0.00802415,  0.00645946, -0.00134831],
       [-0.00804362,  0.0064717 , -0.00133772],
       [-0.00806258,  0.00648379, -0.00132729],
       [-0.008081  ,  0.00649573, -0.00131702],
       [-0.00809887,  0.00650749, -0.00130693],
       [-0.00811616,  0.00651908, -0.00129702],
       [-0.00813285,  0.00653047, -0.00128729],
       [-0.00814893,  0.00654167, -0.00127777],
       [-0.00816437,  0.00655266, -0.00126846],
       [-0.00817915,  0.00656343, -0.00125936],
       [-0.00819326,  0.00657398, -0.0012505 ],
       [-0.00820669,  0.00658429, -0.00124187],
       [-0.00821942,  0.00659436, -0.0012335 ],
       [-0.00823143,  0.00660419, -0.00122538],
       [-0.00824272,  0.00661376, -0.00121754],
       [-0.00825328,  0.00662308, -0.00120998],
       [-0.00826309,  0.00663213, -0.00120271],
       [-0.00827215,  0.00664091, -0.001